# Synthetic COCO Dataset Generation

This notebook generates synthetic images with object annotations in COCO format for training an object detection model. The objects are white shapes (baton, clubs, coin, diamond, heart) that will be detected by a webcam.

## Import Libraries

In [ ]:
import os
import json
import random
import numpy as np
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
from datetime import datetime
from pathlib import Path

## Configuration

In [ ]:
# Configuration
CONFIG = {
    # Classes
    'classes': {
        0: 'background',
        1: 'baton',
        2: 'clubs',
        3: 'coin',
        4: 'diamond',
        5: 'heart'
    },
    
    # Paths to shape templates
    'shape_paths': {
        'baton': 'assets/shapes/baton.png',
        'clubs': 'assets/shapes/clubs.png',
        'coin': 'assets/shapes/coin.png',
        'diamond': 'assets/shapes/diamond.png',
        'heart': 'assets/shapes/heart.png'
    },
    
    # Output settings
    'output_dir': 'output',
    'train_dir': 'train',
    'val_dir': 'val',
    'train_ratio': 0.8,  # 80% train, 20% validation
    
    # Image generation settings
    'num_images': 100,  # Total number of images to generate
    'image_width': 640,
    'image_height': 480,
    'min_objects': 1,
    'max_objects': 3,
    
    # Object transformation settings
    'min_scale': 0.3,
    'max_scale': 0.8,
    'min_rotation': 0,
    'max_rotation': 360,
    
    # Background settings
    'background_complexity': 'high',  # low, medium, high
}

print("Configuration loaded successfully!")

## Helper Functions for Background Generation

In [ ]:
def generate_procedural_background(width, height, complexity='medium'):
    """
    Generate a procedural background with varying complexity.
    
    Args:
        width: Image width
        height: Image height
        complexity: 'low', 'medium', or 'high'
    
    Returns:
        PIL Image
    """
    # Create base with random color
    base_color = tuple(random.randint(100, 255) for _ in range(3))
    img = Image.new('RGB', (width, height), base_color)
    draw = ImageDraw.Draw(img)
    
    if complexity in ['medium', 'high']:
        # Add gradient
        for y in range(height):
            gradient_factor = y / height
            color = tuple(int(c * (1 - gradient_factor * 0.3)) for c in base_color)
            draw.line([(0, y), (width, y)], fill=color)
    
    if complexity == 'high':
        # Add random shapes for texture
        num_shapes = random.randint(5, 15)
        for _ in range(num_shapes):
            shape_type = random.choice(['rectangle', 'ellipse', 'polygon'])
            x1, y1 = random.randint(0, width), random.randint(0, height)
            x2, y2 = x1 + random.randint(50, 200), y1 + random.randint(50, 200)
            
            color = tuple(random.randint(max(0, base_color[i] - 50), 
                                       min(255, base_color[i] + 50)) for i in range(3))
            
            if shape_type == 'rectangle':
                draw.rectangle([x1, y1, x2, y2], fill=color, outline=None)
            elif shape_type == 'ellipse':
                draw.ellipse([x1, y1, x2, y2], fill=color, outline=None)
    
    # Add noise
    noise_level = {'low': 10, 'medium': 20, 'high': 30}[complexity]
    np_img = np.array(img)
    noise = np.random.randint(-noise_level, noise_level, np_img.shape, dtype=np.int16)
    np_img = np.clip(np_img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    img = Image.fromarray(np_img)
    
    # Apply slight blur for realism
    img = img.filter(ImageFilter.GaussianBlur(radius=1))
    
    return img

# Test background generation
test_bg = generate_procedural_background(640, 480, 'medium')
print("Background generation function created successfully!")
test_bg

## Load Shape Templates

In [ ]:
def load_shape_templates(shape_paths):
    """
    Load all shape templates from disk.
    
    Args:
        shape_paths: Dictionary mapping shape names to file paths
    
    Returns:
        Dictionary of shape name to PIL Image
    """
    templates = {}
    for shape_name, path in shape_paths.items():
        if os.path.exists(path):
            img = Image.open(path).convert('RGBA')
            templates[shape_name] = img
            print(f"Loaded {shape_name}: {img.size}")
        else:
            print(f"Warning: {path} not found!")
    
    return templates

# Load templates
shape_templates = load_shape_templates(CONFIG['shape_paths'])
print(f"\nLoaded {len(shape_templates)} shape templates")

## Object Transformation Functions

In [ ]:
def transform_object(img, scale, rotation):
    """
    Apply scale and rotation transformations to an object.
    
    Args:
        img: PIL Image (RGBA)
        scale: Scale factor
        rotation: Rotation angle in degrees
    
    Returns:
        Transformed PIL Image (RGBA)
    """
    # Scale
    new_width = int(img.width * scale)
    new_height = int(img.height * scale)
    img = img.resize((new_width, new_height), Image.LANCZOS)
    
    # Rotate
    img = img.rotate(rotation, expand=True, resample=Image.BICUBIC)
    
    return img


def get_bounding_box(img):
    """
    Get the bounding box of non-transparent pixels in an RGBA image.
    
    Args:
        img: PIL Image (RGBA)
    
    Returns:
        Tuple (x_min, y_min, width, height)
    """
    # Get alpha channel
    alpha = np.array(img.split()[-1])
    
    # Find non-transparent pixels
    rows = np.any(alpha > 0, axis=1)
    cols = np.any(alpha > 0, axis=0)
    
    if not rows.any() or not cols.any():
        return None
    
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    
    width = x_max - x_min + 1
    height = y_max - y_min + 1
    
    return (int(x_min), int(y_min), int(width), int(height))


def place_object_on_background(background, obj_img, position):
    """
    Place an object on a background at a given position.
    
    Args:
        background: PIL Image (RGB)
        obj_img: PIL Image (RGBA)
        position: Tuple (x, y) for top-left corner
    
    Returns:
        Modified background image and bounding box relative to background
    """
    # Get object bounding box
    obj_bbox = get_bounding_box(obj_img)
    if obj_bbox is None:
        return background, None
    
    # Paste object on background
    background.paste(obj_img, position, obj_img)
    
    # Calculate bounding box relative to background
    x, y = position
    obj_x, obj_y, obj_w, obj_h = obj_bbox
    
    bbox = (x + obj_x, y + obj_y, obj_w, obj_h)
    
    return background, bbox

print("Transformation functions created successfully!")

## COCO Dataset Structure

In [ ]:
def create_coco_structure():
    """
    Create the initial COCO dataset structure.
    
    Returns:
        Dictionary with COCO format structure
    """
    # Get category information from CONFIG
    categories = []
    for cat_id, cat_name in CONFIG['classes'].items():
        # Include all categories, including background (category 0)
        categories.append({
            'id': cat_id,
            'name': cat_name,
            'supercategory': 'shape'
        })
    
    coco_structure = {
        'info': {
            'description': 'Synthetic Object Detection Dataset',
            'version': '1.0',
            'year': 2025,
            'date_created': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        },
        'licenses': [],
        'categories': categories,
        'images': [],
        'annotations': []
    }
    
    return coco_structure


def add_image_to_coco(coco_dict, image_id, file_name, width, height):
    """
    Add an image entry to COCO structure.
    """
    image_entry = {
        'id': image_id,
        'file_name': file_name,
        'width': width,
        'height': height,
        'date_captured': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    coco_dict['images'].append(image_entry)


def add_annotation_to_coco(coco_dict, annotation_id, image_id, category_id, bbox):
    """
    Add an annotation entry to COCO structure.
    
    Args:
        coco_dict: COCO structure dictionary
        annotation_id: Unique annotation ID
        image_id: ID of the image this annotation belongs to
        category_id: Category ID (class)
        bbox: Tuple (x, y, width, height) in pixels
    """
    x, y, w, h = bbox
    area = w * h
    
    annotation = {
        'id': annotation_id,
        'image_id': image_id,
        'category_id': category_id,
        'bbox': [float(x), float(y), float(w), float(h)],
        'area': float(area),
        'iscrowd': 0
    }
    coco_dict['annotations'].append(annotation)

print("COCO structure functions created successfully!")

## Image Generation Function

In [ ]:
def generate_synthetic_image(templates, config):
    """
    Generate a single synthetic image with random objects.
    
    Args:
        templates: Dictionary of shape templates
        config: Configuration dictionary
    
    Returns:
        Tuple (PIL Image, list of annotations)
        Annotations: list of dicts with 'category_id' and 'bbox'
    """
    # Create background
    background = generate_procedural_background(
        config['image_width'],
        config['image_height'],
        config['background_complexity']
    )
    
    # Determine number of objects
    num_objects = random.randint(config['min_objects'], config['max_objects'])
    
    # List to store annotations
    annotations = []
    
    # Get list of available shape names (excluding background)
    shape_names = [name for name in templates.keys()]
    
    for _ in range(num_objects):
        # Select random shape
        shape_name = random.choice(shape_names)
        shape_img = templates[shape_name].copy()
        
        # Get category_id from class name
        category_id = None
        for cat_id, cat_name in config['classes'].items():
            if cat_name == shape_name:
                category_id = cat_id
                break
        
        if category_id is None:
            continue
        
        # Random transformations
        scale = random.uniform(config['min_scale'], config['max_scale'])
        rotation = random.uniform(config['min_rotation'], config['max_rotation'])
        
        # Transform object
        transformed_obj = transform_object(shape_img, scale, rotation)
        
        # Random position (ensure object fits in image)
        max_x = max(0, config['image_width'] - transformed_obj.width)
        max_y = max(0, config['image_height'] - transformed_obj.height)
        
        if max_x <= 0 or max_y <= 0:
            continue
        
        position = (random.randint(0, max_x), random.randint(0, max_y))
        
        # Place object on background
        background, bbox = place_object_on_background(background, transformed_obj, position)
        
        if bbox is not None:
            # Store annotation
            annotations.append({
                'category_id': category_id,
                'bbox': bbox
            })
    
    return background, annotations

print("Image generation function created successfully!")

## Test Image Generation

In [ ]:
# Generate a test image to verify everything works
if shape_templates:
    test_img, test_annotations = generate_synthetic_image(shape_templates, CONFIG)
    print(f"Generated test image with {len(test_annotations)} objects")
    for ann in test_annotations:
        cat_name = CONFIG['classes'][ann['category_id']]
        print(f"  - {cat_name}: bbox {ann['bbox']}")
    
    # Display the test image
    display(test_img)
else:
    print("No shape templates loaded. Please check the paths in CONFIG.")

## Main Dataset Generation

In [ ]:
def generate_dataset(templates, config):
    """
    Generate the complete synthetic dataset.
    
    Args:
        templates: Dictionary of shape templates
        config: Configuration dictionary
    """
    # Create output directories
    output_dir = Path(config['output_dir'])
    train_dir = output_dir / config['train_dir']
    val_dir = output_dir / config['val_dir']
    
    train_images_dir = train_dir / 'images'
    val_images_dir = val_dir / 'images'
    
    train_images_dir.mkdir(parents=True, exist_ok=True)
    val_images_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Output directories created:")
    print(f"  - {train_images_dir}")
    print(f"  - {val_images_dir}")
    
    # Initialize COCO structures
    train_coco = create_coco_structure()
    val_coco = create_coco_structure()
    
    # Calculate split
    num_train = int(config['num_images'] * config['train_ratio'])
    num_val = config['num_images'] - num_train
    
    print(f"\nGenerating {config['num_images']} images:")
    print(f"  - Training: {num_train}")
    print(f"  - Validation: {num_val}")
    
    # Counters
    annotation_id = 1
    
    # Generate training images
    print("\nGenerating training images...")
    for i in range(num_train):
        # Generate image
        img, annotations = generate_synthetic_image(templates, config)
        
        # Save image
        image_id = i + 1
        file_name = f"train_{image_id:06d}.jpg"
        img_path = train_images_dir / file_name
        img.save(img_path, 'JPEG', quality=95)
        
        # Add to COCO structure
        add_image_to_coco(train_coco, image_id, file_name, 
                         config['image_width'], config['image_height'])
        
        for ann in annotations:
            add_annotation_to_coco(train_coco, annotation_id, image_id,
                                  ann['category_id'], ann['bbox'])
            annotation_id += 1
        
        if (i + 1) % 100 == 0:
            print(f"  Generated {i + 1}/{num_train} training images")
    
    # Generate validation images
    print("\nGenerating validation images...")
    annotation_id = 1  # Reset for validation
    for i in range(num_val):
        # Generate image
        img, annotations = generate_synthetic_image(templates, config)
        
        # Save image
        image_id = i + 1
        file_name = f"val_{image_id:06d}.jpg"
        img_path = val_images_dir / file_name
        img.save(img_path, 'JPEG', quality=95)
        
        # Add to COCO structure
        add_image_to_coco(val_coco, image_id, file_name,
                         config['image_width'], config['image_height'])
        
        for ann in annotations:
            add_annotation_to_coco(val_coco, annotation_id, image_id,
                                  ann['category_id'], ann['bbox'])
            annotation_id += 1
        
        if (i + 1) % 100 == 0 or (i + 1) == num_val:
            print(f"  Generated {i + 1}/{num_val} validation images")
    
    # Save COCO annotations
    train_annotations_path = train_dir / 'labels.json'
    val_annotations_path = val_dir / 'labels.json'
    
    with open(train_annotations_path, 'w') as f:
        json.dump(train_coco, f, indent=2)
    
    with open(val_annotations_path, 'w') as f:
        json.dump(val_coco, f, indent=2)
    
    # Save configuration for reference during training/evaluation
    config_path = output_dir / 'config.json'
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"\nAnnotations saved:")
    print(f"  - {train_annotations_path}")
    print(f"  - {val_annotations_path}")
    print(f"\nConfiguration saved:")
    print(f"  - {config_path}")
    
    # Print statistics
    print(f"\nDataset Statistics:")
    print(f"  Training images: {len(train_coco['images'])}")
    print(f"  Training annotations: {len(train_coco['annotations'])}")
    print(f"  Validation images: {len(val_coco['images'])}")
    print(f"  Validation annotations: {len(val_coco['annotations'])}")
    print(f"  Classes: {len(train_coco['categories'])}")
    
    return train_coco, val_coco

print("Dataset generation function ready!")

## Load Configuration

Helper function to load the saved configuration from a dataset directory.

In [ ]:
def load_dataset_config(dataset_dir):
    """
    Load the configuration file from a dataset directory.
    
    Args:
        dataset_dir: Path to the dataset output directory
    
    Returns:
        Dictionary with configuration parameters
    """
    config_path = Path(dataset_dir) / 'config.json'
    
    if not config_path.exists():
        raise FileNotFoundError(f"Configuration file not found: {config_path}")
    
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    return config

print("Configuration loader function ready!")

## Generate the Dataset

Run this cell to generate the complete synthetic dataset. Adjust the `num_images` in CONFIG if needed.

In [ ]:
# Generate the dataset
if shape_templates:
    train_coco, val_coco = generate_dataset(shape_templates, CONFIG)
    print("\n✅ Dataset generation complete!")
else:
    print("❌ Cannot generate dataset: No shape templates loaded.")
    print("Please check that the shape images exist in the assets/shapes/ directory.")

## Visualize Sample Images

Let's visualize a few generated images with their bounding boxes to verify the dataset quality.

In [ ]:
def visualize_annotations(image_path, annotations, categories):
    """
    Visualize an image with its bounding box annotations.
    
    Args:
        image_path: Path to the image
        annotations: List of annotation dictionaries
        categories: List of category dictionaries from COCO format
    
    Returns:
        PIL Image with bounding boxes drawn
    """
    # Load image
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    
    # Create category lookup
    cat_lookup = {cat['id']: cat['name'] for cat in categories}
    
    # Colors for different classes
    colors = {
        'baton': 'red',
        'clubs': 'green',
        'coin': 'blue',
        'diamond': 'yellow',
        'heart': 'magenta'
    }
    
    # Draw bounding boxes
    for ann in annotations:
        x, y, w, h = ann['bbox']
        cat_name = cat_lookup[ann['category_id']]
        color = colors.get(cat_name, 'white')
        
        # Draw rectangle
        draw.rectangle([x, y, x + w, y + h], outline=color, width=3)
        
        # Draw label
        draw.text((x, y - 15), cat_name, fill=color)
    
    return img


# Visualize a few training images
if shape_templates and 'train_coco' in locals():
    print("Sample Training Images with Annotations:\n")
    
    train_dir = Path(CONFIG['output_dir']) / CONFIG['train_dir']
    
    # Show 3 random samples
    sample_indices = random.sample(range(len(train_coco['images'])), min(3, len(train_coco['images'])))
    
    for idx in sample_indices:
        img_info = train_coco['images'][idx]
        img_path = train_dir / 'images' / img_info['file_name']
        
        # Get annotations for this image
        img_annotations = [ann for ann in train_coco['annotations'] 
                          if ann['image_id'] == img_info['id']]
        
        print(f"\n{img_info['file_name']}: {len(img_annotations)} objects")
        
        # Visualize
        vis_img = visualize_annotations(img_path, img_annotations, train_coco['categories'])
        display(vis_img)
else:
    print("Dataset not yet generated. Run the generation cell first.")

## Dataset Statistics and Analysis

In [ ]:
if shape_templates and 'train_coco' in locals() and 'val_coco' in locals():
    # Analyze class distribution
    from collections import Counter
    
    print("=" * 60)
    print("DATASET ANALYSIS")
    print("=" * 60)
    
    # Training set analysis
    print("\n📊 TRAINING SET:")
    print(f"  Total images: {len(train_coco['images'])}")
    print(f"  Total annotations: {len(train_coco['annotations'])}")
    
    train_class_counts = Counter([ann['category_id'] for ann in train_coco['annotations']])
    print(f"\n  Class distribution:")
    for cat in train_coco['categories']:
        count = train_class_counts[cat['id']]
        print(f"    - {cat['name']}: {count} instances")
    
    # Validation set analysis
    print("\n📊 VALIDATION SET:")
    print(f"  Total images: {len(val_coco['images'])}")
    print(f"  Total annotations: {len(val_coco['annotations'])}")
    
    val_class_counts = Counter([ann['category_id'] for ann in val_coco['annotations']])
    print(f"\n  Class distribution:")
    for cat in val_coco['categories']:
        count = val_class_counts[cat['id']]
        print(f"    - {cat['name']}: {count} instances")
    
    # Average objects per image
    avg_train_obj = len(train_coco['annotations']) / len(train_coco['images'])
    avg_val_obj = len(val_coco['annotations']) / len(val_coco['images'])
    
    print(f"\n📈 STATISTICS:")
    print(f"  Average objects per training image: {avg_train_obj:.2f}")
    print(f"  Average objects per validation image: {avg_val_obj:.2f}")
    
    print("\n" + "=" * 60)
    print("✅ Dataset ready for training!")
    print("=" * 60)
else:
    print("Dataset not yet generated. Run the generation cell first.")

## Next Steps

After generating the dataset:

1. **Review the generated images** in the `output/train/images/` and `output/val/images/` directories
2. **Check the annotations** in `output/train/labels.json` and `output/val/labels.json`
3. **Train your MediaPipe object detector** using this synthetic dataset
4. **Fine-tune parameters** if needed:
   - Adjust `num_images` for more training data
   - Modify `min_objects` and `max_objects` for different object densities
   - Change `background_complexity` for varied backgrounds
   - Adjust scale and rotation ranges for different augmentations

The dataset is now ready to be used with MediaPipe Object Detector training pipeline!